In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver as wire_webdriver
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
import dbm

In [18]:
from datetime import datetime

In [44]:
from datetime import date

In [2]:
# Declare browser service
s = Service('C://Users//hantt3//chromedriver_win32//chromedriver.exe')

# Configure Chrome options
options = Options()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')

# Create the webdriver instance
driver = wire_webdriver.Chrome(service=s, options=options)

driver.get('https://www.lazada.sg/#')

In [3]:
#click on the flash sale section to scrape products is for flash sale
flashsale = driver.find_element(By.CLASS_NAME, 'card-fs-content-button')
flashsale.click()

In [4]:
# Define a variable to track the presence of the "Load More" button
load_more_present = True

while load_more_present:
    try:
        # Find all "Load More" buttons on the page
        load_more_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.button.J_LoadMoreButton[data-camp-index="0"]')

        # If no "Load More" button is found, break out of the loop
        if len(load_more_buttons) == 0:
            break

        # Iterate through each button and click on it
        for button in load_more_buttons:
            button.click()

        # Wait for some time (if needed) to allow the new items to load
        time.sleep(3)  # Adjust the sleep duration as needed

    except NoSuchElementException:
        # If no "Load More" button is found, set the variable to False to exit the loop
        load_more_present = False


In [11]:
title, sale_price, origin_price, discount, image = [], [], [], [], []
product_containers = driver.find_elements(By.CSS_SELECTOR,'.flash-unit.card-hover.pull-left.false.no-margin-right')
for container in product_containers:
    titles = container.find_elements( By.CLASS_NAME, 'sale-title')
    title += [s.text for s in titles] 
    sale_prices = container.find_elements(By.CLASS_NAME, 'sale-price')
    sale_price += [s.text for s in sale_prices]
    origin_prices = container.find_elements(By.CLASS_NAME, 'origin-price-value')
    origin_price += [s.text for s in origin_prices]
    discounts = container.find_elements(By.CLASS_NAME, 'discount')
    discount += [s.text for s in discounts ]
    image_elements = container.find_elements(By.CSS_SELECTOR, 'div.flash-unit-image img.image')
    image += [s.get_attribute("src") for s in image_elements]


In [12]:
df = pd.DataFrame(list(zip(title, sale_price, origin_price, discount, image)), 
                   columns = ['title', 'sale_price','origin_price', 'discount', 'image'])

In [19]:
# Get the current datetime
current_datetime = datetime.today()

# Add a new column with the current datetime
df['Timestamp'] = current_datetime

In [47]:
df = df.drop('Timestamp', axis=1)

In [45]:
current_date = date.today()

df['Date'] = current_date

In [48]:
df.tail()

,title,sale_price,origin_price,discount,image,Date
71,[EXCLUSIVE SALES] AKEMI TENCEL™ Modal Ardent 8...,69.30,99.00,-30,https://lzd-img-global.slatic.net/g/p/6983b5f2...,2023-06-27
72,Sunveno Baby Stroller Organizer Bag Infant Pra...,16.82,31.71,-47,https://lzd-img-global.slatic.net/g/p/5b2295cd...,2023-06-27
73,Simplus Air Fryer 5L/3.5L Nonstick Detachable ...,45.90,139.80,-67,https://lzd-img-global.slatic.net/g/p/7202ac5c...,2023-06-27
74,Kiribai/ Kincho Heat Pack/ Heat pad/ Hand Warm...,9.50,10.90,-13,https://lzd-img-global.slatic.net/g/p/1356ff62...,2023-06-27
75,Chuga Jeju Series Plum Wine Soju,8.50,15.04,-43,https://lzd-img-global.slatic.net/g/p/2a8c4546...,2023-06-27


In [22]:
df.dtypes

title                   object
sale_price              object
origin_price            object
discount                object
image                   object
Timestamp       datetime64[ns]
dtype: object

In [24]:
# Clean and convert the "sale_price", 'origin_price' column to numeric values
df['sale_price'] = df['sale_price'].str.replace('$', '')  # Remove the dollar sign
df['origin_price'] = df['origin_price'].str.replace('$', '')
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')  # Convert to numeric values
df['origin_price'] = pd.to_numeric(df['origin_price'], errors='coerce')

# Clean and convert the "discount" column to numeric values
df['discount'] = df['discount'].str.replace('%', '')  # Remove the percentage sign
df['discount'] = pd.to_numeric(df['discount'], errors='coerce')  # Convert to numeric values

C:\Users\hantt3\AppData\Local\Temp\ipykernel_12900\3241118235.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sale_price'] = df['sale_price'].str.replace('$', '')  # Remove the dollar sign
C:\Users\hantt3\AppData\Local\Temp\ipykernel_12900\3241118235.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['origin_price'] = df['origin_price'].str.replace('$', '')


In [25]:
df.dtypes

title                   object
sale_price             float64
origin_price           float64
discount                 int64
image                   object
Timestamp       datetime64[ns]
dtype: object

In [27]:
# Find the index of the row with the highest price in the "origin" column
max_price_index = df['origin_price'].idxmax()

# Get the product with the highest price
product_with_highest_price = df.loc[max_price_index]

product_with_highest_price


title           Flagship | Dibea Diamond DC22 3-in-1 Wet Dry C...
sale_price                                                  279.0
origin_price                                                853.0
discount                                                      -67
image           https://lzd-img-global.slatic.net/g/p/740f9514...
Timestamp                              2023-06-27 11:52:04.836948
Name: 24, dtype: object

In [29]:
df.tail()

,title,sale_price,origin_price,discount,image,Timestamp
71,[EXCLUSIVE SALES] AKEMI TENCEL™ Modal Ardent 8...,69.30,99.00,-30,https://lzd-img-global.slatic.net/g/p/6983b5f2...,2023-06-27 11:52:04.836948
72,Sunveno Baby Stroller Organizer Bag Infant Pra...,16.82,31.71,-47,https://lzd-img-global.slatic.net/g/p/5b2295cd...,2023-06-27 11:52:04.836948
73,Simplus Air Fryer 5L/3.5L Nonstick Detachable ...,45.90,139.80,-67,https://lzd-img-global.slatic.net/g/p/7202ac5c...,2023-06-27 11:52:04.836948
74,Kiribai/ Kincho Heat Pack/ Heat pad/ Hand Warm...,9.50,10.90,-13,https://lzd-img-global.slatic.net/g/p/1356ff62...,2023-06-27 11:52:04.836948
75,Chuga Jeju Series Plum Wine Soju,8.50,15.04,-43,https://lzd-img-global.slatic.net/g/p/2a8c4546...,2023-06-27 11:52:04.836948


## Push data into MYSQL 

In [28]:
import mysql.connector

In [40]:
!pip install pymysql

     -------------------------------------- 44.8/44.8 kB 750.6 kB/s eta 0:00:00


In [42]:
# Import pymysql module
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='12345678',
                             db='db')
                             
cursor = connection.cursor()

In [73]:
sql = "CREATE TABLE IF NOT EXISTS lazada_flashsale(title VARCHAR(255) DEFAULT NULL, sale_price FLOAT DEFAULT NULL, origin_price FLOAT DEFAULT NULL, discount FLOAT DEFAULT NULL, image VARCHAR(255) DEFAULT NULL, Date date)"

# Executing Query
cursor.execute(sql)

# Commit the Query
connection.commit()

In [63]:
from sqlalchemy import create_engine

In [64]:
engine = create_engine('mysql+pymysql://', creator=lambda: connection)

In [65]:
table_name = 'lazada_flashsale'

In [74]:
df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

76